In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

from PIL import Image
import os

In [2]:
#import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
# For moving all img files from one root dir to another
# source = "Train"
# dest = "Train1"

# label_flood_dir = os.path.join('Labeled','Flooded','image')
# label_nonflood_dir = os.path.join('Labeled','Non-Flooded','image')

# for dire in label_flood_dir,label_nonflood_dir:
#     source_dir = os.path.join(source,dire)
#     dest_dir = os.path.join(dest,dire)
#     for file in os.listdir(source_dir):
#         source_file = os.path.join(source_dir,file)
#         dest_file = os.path.join(dest_dir,file)
#         os.rename(source_file,dest_file)

Should take around 3 minutes to load

In [6]:
h_dim = 1000
v_dim = 750
print("Loading Data from Memory")

root = "../Train1"
label_flood_dir = os.path.join(root,'Labeled','Flooded','image')
label_nonflood_dir = os.path.join(root,'Labeled','Non-Flooded','image')
flooded_img = []
nonflooded_img = []

for file in os.listdir(label_flood_dir):
    image = Image.open(os.path.join(label_flood_dir, file))
    flooded_img.append(np.array(image.resize((h_dim,v_dim))))
    
for file in os.listdir(label_nonflood_dir):
    image = Image.open(os.path.join(label_nonflood_dir, file))
    nonflooded_img.append(np.array(image.resize((h_dim,v_dim))))
print("Loaded!")

Loading Data from Memory
Loaded!


In [7]:
print("Flooded Image Shape: {}".format(flooded_img[0].shape))
print("Non_Flooded Image Shape: {}".format(nonflooded_img[0].shape))

data_img = np.vstack((np.array(flooded_img), np.array(nonflooded_img))) / 255.
data_img.shape

Flooded Image Shape: (750, 1000, 3)
Non_Flooded Image Shape: (750, 1000, 3)


(200, 750, 1000, 3)

In [ ]:
def unbalanced_tts(flooded_img, nonflooded_img, unlabelled_img=np.array([])):
    for 

In [6]:
import random
test = False

#n is number images from each class (flooded or non flooded)
if test == True:
    n = 20
else:
    n = min(len(flooded_img),len(nonflooded_img))

'''split data 50 50'''
def train_test_split(n):
    idxs = list(range(n))
    s = int(np.floor(0.8*n)) # number of images for training
    
    train_idx = []
    test_idx = []
    
    #index range to select from (flooded img range, non flooded image range)
    for s_i,e_i in [(0,len(flooded_img)),(len(flooded_img),len(data_img))]:
        
        #print(s_i,e_i,n,s)
        #get all poss indexes for set
        s_idx = list(range(s_i,e_i))
        random.shuffle(s_idx)
        
        train_idx.extend(s_idx[:s])      #first s images are for training
        test_idx.extend(s_idx[s:n])      # next n-s images are for testing. 
        #print(len(train_idx))
    
    train_idx = np.array(train_idx)
    test_idx = np.array(test_idx)
    train_idx.sort()
    test_idx.sort()
    
    train_labels = [1 if x<len(flooded_img) else 0 for x in train_idx]
    test_labels = [1 if x<len(flooded_img) else 0 for x in test_idx]

    print("Training Index: {}".format(train_idx))
    print("Testing Index: {}".format(test_idx))
    return train_idx, test_idx, train_labels, test_labels

train_idx, test_idx, train_labels, test_labels = train_test_split(n)


Training Index: [  0   3   4   5   7   9  10  12  13  14  15  17  18  19  20  21  22  23
  24  25  26  28  29  30  31  33  34  35  36  37  38  39  41  42  43  44
  45  46  47  48  54  57  75  85  96  98  99 102 109 115 118 120 125 140
 146 186 187 203 207 208 209 229 256 265 268 282 297 301 305 307 310 321
 332 343 345 347 361 369 380 389]
Testing Index: [  1   2   6   8  11  16  27  32  40  49  50  58  60 154 170 171 269 284
 350 375 381 386]


In [7]:
# convolutional layer
# x the input 
# shape is dimension of input
def conv_layer(x, shape):

    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
    bias = tf.Variable(tf.constant(0.05, shape=[shape[-1]]))

    out = tf.nn.conv2d(input=x, filters=weights, strides=[1,1,1,1], padding='SAME')
    out += bias
    return out

# pooling layer
def max_pool(x, k=2):

    out = tf.nn.max_pool(value=x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')
    return out

# fully connected layer
def fully_connected_layer(x, shape):

    weights = tf.Variable(tf.truncated_normal(shape, stddev=0.05))
    bias = tf.Variable(tf.constant(0.05, shape=[shape[1]]))

    out = tf.matmul(a=x, b=weights)
    out += bias
    return out

# flatten layer
def flatten_layer(x):
    
    size = x.get_shape()[1:4].num_elements()
    out = tf.reshape(x, [-1,size])
    return out, size

# relu
relu = lambda x: tf.nn.relu(features=x)

# softmax
softmax = lambda x: tf.nn.softmax(logits=x)

# sigmoid
sigmoid = lambda x: tf.nn.sigmoid(x)

# batch norm
batch_norm = lambda x: tf.layers.batch_normalization(x)

In [8]:
# define CNN
def toy_model(x):
    
    #shape = [filter_size, filter_size, num_input_channels, num_filters]
    def conv_block(x,in_channels,out_channels,kernel_h=5,kernel_w =5):
        shape = [kernel_h,kernel_w,in_channels,out_channels]
        x = conv_layer(x, shape)
        x = relu(x)
        x = batch_norm(x)
        x = max_pool(x, k=2)
        return x
    
    x = conv_block(x,3,16)
    x = conv_block(x,16,16)
    x = conv_block(x,16,32)
    x = conv_block(x,32,32)
    x = conv_block(x,32,64)
    x = conv_block(x,64,64)
    x = conv_block(x,64,1)

    # flatten output and put through a fully connected layer
    flat1, size1 = flatten_layer(x)
    fc1 = fully_connected_layer(flat1, [size1, 64])
    fc1 = relu(fc1)

    fc2 = fully_connected_layer(fc1, [64, 1])

    return fc2

In [9]:
# def sharpen(p):
#     T = 0.5
#     pred = p**(1./T)/(p**(1./T) + (1.-p)**(1./T))
#     return pred

In [10]:
# define inputs
x = tf.placeholder(tf.float32, [None, v_dim, h_dim, 3])
y = tf.placeholder(tf.float32, [None, 1])
y_train_true = np.array(train_labels).reshape(-1,1)
y_test_true = np.array(test_labels).reshape(-1,1)

# run model with placeholder tensors
pred = toy_model(x)

# sharpen
# pred = sharpen(pred)

# define loss
cross_entropy = tf.losses.sigmoid_cross_entropy(logits=pred, multi_class_labels=y)
cost = tf.reduce_mean(cross_entropy)

# define accuracy
pred_class = tf.round(sigmoid(pred))
pred_correct = tf.equal(pred_class, tf.cast(y, tf.float32))
accuracy = tf.reduce_mean(tf.cast(pred_correct, tf.float32))

# define optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=1e-2).minimize(cost)

# initialize variables
init = tf.global_variables_initializer()
training_iters = 500
batch_size = 4 #len(train_idx)

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
# train model
with tf.Session(config=config) as sess:
    sess.run(init)
    train_loss = []
    test_loss = []
    train_accuracy = []
    test_accuracy = []
    best_acc = 0.
    summary_writer = tf.summary.FileWriter('./Output', sess.graph)
    num_batches = len(train_idx)//batch_size
    
    for i in range(training_iters):
        
        # Reset metrics
        loss_total = 0
        acc_total = 0
        train_results = []
   
        # Run optimization 
        # Calculate batch loss and accuracy
        for batch in range(num_batches):
            batch_x = data_img[train_idx,:,:,:][batch*batch_size:min((batch+1)*batch_size,len(train_idx))]
            batch_y = y_train_true[batch*batch_size:min((batch+1)*batch_size,len(y_train_true))]    

            feed_dict={x: batch_x, y: batch_y}
            opt = sess.run(optimizer, feed_dict=feed_dict)
            loss, acc, pred_labels = sess.run([cost, accuracy, pred_class], feed_dict=feed_dict)
            loss_total += loss
            acc_total += acc
            train_results.append(pred_labels)

        # Average metrics
        ave_loss = loss_total/num_batches
        ave_acc = acc_total/num_batches

        # Calculate accuracy for all test images
        valid_loss, test_acc, test_results = sess.run([cost, accuracy, pred_class],
                                feed_dict={x: data_img[test_idx,:,:,:], y : y_test_true})
        
        # Update metrics
        train_loss.append(ave_loss)
        test_loss.append(valid_loss)
        train_accuracy.append(ave_acc)
        test_accuracy.append(test_acc)
        if test_acc > best_acc:
            best_model_train_labels = tf.stack(tf.reshape(tf.stack(train_results),[-1,1])).eval()
            best_model_test_labels = test_results
            best_acc = test_acc
        
        # Print metrics
        print("Iter " + str(i) + ", Loss= " + \
                      "{:.6f}".format(ave_loss) + ", Training Accuracy= " + \
                      "{:.5f}".format(ave_acc)+ \
                      " ,Testing Accuracy:","{:.5f}".format(test_acc))
    summary_writer.close()

Iter 0, Loss= 5037.220654, Training Accuracy= 0.75000 ,Testing Accuracy: 0.50000
Iter 1, Loss= 0.781362, Training Accuracy= 0.50000 ,Testing Accuracy: 0.50000
Iter 2, Loss= 0.706371, Training Accuracy= 0.50000 ,Testing Accuracy: 0.50000
Iter 3, Loss= 0.701583, Training Accuracy= 0.40000 ,Testing Accuracy: 0.50000
Iter 4, Loss= 0.700327, Training Accuracy= 0.35000 ,Testing Accuracy: 0.50000
Iter 5, Loss= 0.699717, Training Accuracy= 0.30000 ,Testing Accuracy: 0.50000
Iter 6, Loss= 0.699513, Training Accuracy= 0.25000 ,Testing Accuracy: 0.50000
Iter 7, Loss= 0.699023, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 8, Loss= 0.698947, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 9, Loss= 0.670100, Training Accuracy= 0.40000 ,Testing Accuracy: 0.50000
Iter 10, Loss= 0.731155, Training Accuracy= 0.50000 ,Testing Accuracy: 0.50000
Iter 11, Loss= 0.780346, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 12, Loss= 0.717637, Training Accuracy= 0.05000 ,Testin

Iter 104, Loss= 0.698437, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 105, Loss= 0.698409, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 106, Loss= 0.698382, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 107, Loss= 0.698356, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 108, Loss= 0.698331, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 109, Loss= 0.698307, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 110, Loss= 0.698283, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 111, Loss= 0.698260, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 112, Loss= 0.698238, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 113, Loss= 0.698216, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 114, Loss= 0.698195, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 115, Loss= 0.698175, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 116, Loss= 0.698155, Training Accur

Iter 207, Loss= 0.696992, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 208, Loss= 0.696984, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 209, Loss= 0.696976, Training Accuracy= 0.15000 ,Testing Accuracy: 0.50000
Iter 210, Loss= 0.696969, Training Accuracy= 0.15000 ,Testing Accuracy: 0.50000
Iter 211, Loss= 0.696962, Training Accuracy= 0.15000 ,Testing Accuracy: 0.50000
Iter 212, Loss= 0.696954, Training Accuracy= 0.15000 ,Testing Accuracy: 0.50000
Iter 213, Loss= 0.696941, Training Accuracy= 0.15000 ,Testing Accuracy: 0.50000
Iter 214, Loss= 0.697003, Training Accuracy= 0.15000 ,Testing Accuracy: 0.50000
Iter 215, Loss= 0.696963, Training Accuracy= 0.15000 ,Testing Accuracy: 0.50000
Iter 216, Loss= 0.696952, Training Accuracy= 0.15000 ,Testing Accuracy: 0.50000
Iter 217, Loss= 0.696945, Training Accuracy= 0.15000 ,Testing Accuracy: 0.50000
Iter 218, Loss= 0.696938, Training Accuracy= 0.15000 ,Testing Accuracy: 0.50000
Iter 219, Loss= 0.696931, Training Accur

Iter 310, Loss= 0.696557, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 311, Loss= 0.696555, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 312, Loss= 0.696553, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 313, Loss= 0.696552, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 314, Loss= 0.696550, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 315, Loss= 0.696549, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 316, Loss= 0.696548, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 317, Loss= 0.696546, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 318, Loss= 0.696546, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 319, Loss= 0.696545, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 320, Loss= 0.696544, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 321, Loss= 0.696543, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 322, Loss= 0.696543, Training Accur

Iter 413, Loss= 0.696539, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 414, Loss= 0.696539, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 415, Loss= 0.696539, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 416, Loss= 0.696539, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 417, Loss= 0.696539, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 418, Loss= 0.696539, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 419, Loss= 0.696539, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 420, Loss= 0.696539, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 421, Loss= 0.696539, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 422, Loss= 0.696539, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 423, Loss= 0.696539, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 424, Loss= 0.696539, Training Accuracy= 0.20000 ,Testing Accuracy: 0.50000
Iter 425, Loss= 0.696539, Training Accur

In [ ]:
best_model_train_labels

In [ ]:
best_model_test_labels

In [ ]:
for title, data in {"Loss":train_loss,"Train Accuracy": train_accuracy, "Test Accuracy": test_accuracy}.items():
    plt.plot(data)
    plt.title(title)
    plt.show()